In [1]:
import os, sys; sys.path.append("utils")
from constants import *

import numpy as np
import pandas as pd

from sklearn.metrics import cohen_kappa_score as kappa


aspects = ["consistency", "coherence", "fluency", "relevance"]

In [3]:
for model in ["mistral", "llama2", "llama3"]:
    results = pd.DataFrame(columns=["dataset"]+aspects)
    for dataset in ["summeval", "newsroom"]:
        row = [dataset]
        for aspect in aspects:
            path = f"{gdrive_path}/logits_short/{model}/{dataset}_{aspect}.jsonl"
            logits = pd.read_json(path, orient="records", lines=True)
            accs, ks = [], []
            for article in logits["article_id"].unique():
                subset = logits.loc[logits["article_id"] == article, [aspect, "p_s1"]].copy()
                subset[aspect] = subset[aspect].apply(lambda x: (x[0] > x[1])*1)
                subset["p_s1"] = subset["p_s1"].apply(lambda x: (x > 0.5)*1)
                acc = (subset[aspect] == subset["p_s1"]).mean()
                k = kappa(subset[aspect], subset["p_s1"])
                accs.append(acc)
                ks.append(k)
            row.extend([np.mean(ks)])
        results.loc[len(results)] = row
    results.set_index("dataset", inplace=True)
    print(model)
    display(results)

mistral


,consistency,coherence,fluency,relevance
dataset,,,,
summeval,0.077988,0.123817,0.071252,0.176896
newsroom,0.072274,0.048968,0.092415,0.144902


llama2


,consistency,coherence,fluency,relevance
dataset,,,,
summeval,0.108530,0.157500,0.047572,0.159315
newsroom,0.261633,0.258971,0.193046,0.276952


llama3


,consistency,coherence,fluency,relevance
dataset,,,,
summeval,0.148657,0.263699,0.167664,0.266690
newsroom,0.355684,0.398756,0.361434,0.359748
